<a href="https://colab.research.google.com/github/DavidSchneider47/ALIrecommendation/blob/main/TOD_Shapefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import some libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

In [5]:
# read in the file
data = pd.read_csv('stations1.csv')

In [6]:
#show the first few rows
data.head()

,Lon,Lat,Facility Name,Facility Type,Primary Mode Served,Street Address,City,State
0,-150.06188,62.18922,Talkeetna Depot,At-Grade Fixed Guideway Station,AR,Mile 13.7 Talkeetna Spur Rd.,Talkeetna,AK
1,-149.58490,61.10283,Anchorage Airport Bill Sheffield Depot,Elevated Fixed Guideway Station,AR,4980 W Int'L Airport Rd.,Anchorage,AK
2,-149.53262,61.13180,Anchorage Historic Depot (all inclusive),At-Grade Fixed Guideway Station,AR,401 - 431 1St. Ave.,Anchorage,AK
3,-149.26326,60.07287,Seward Depot,At-Grade Fixed Guideway Station,AR,408 Port Ave.,Seward,AK
4,-149.10306,60.56418,Girdwood Depot,At-Grade Fixed Guideway Station,AR,Toadstool Dr,Girdwood,AK


In [10]:
#identify and remove the null values
num_null_values = data['Lon'].isnull().sum()
num_null_values

233

In [12]:
#drop the null values and update the original dataset
data.dropna(subset=['Lon'], inplace=True)

In [15]:
#identify duplicate lon/lat coordinates
num_duplicates = data.duplicated(subset=['Lat']).sum()
num_duplicates

14

In [33]:
# Create a GeoDataFrame from your data
geometry = [Point(xy) for xy in zip(data.Lon, data.Lat)]
geo_df = gpd.GeoDataFrame(data, geometry=geometry)

In [34]:
# Define the CRS (coordinate reference system)
# You need to define it because operations like buffer depend on the unit of measurement,
# and lat/lon is often represented in degrees.
geo_df.crs = CRS("EPSG:4326")  # This is the CRS for latitude/longitude
geo_df = geo_df.to_crs("EPSG:3857")  # Converting to a CRS that uses meters as units (Web Mercator)

In [35]:
# Now, create a buffer of 1/4 mile around each point.
# Since EPSG:3857 is in meters and 1/4 mile is about 402.336 meters, we use that as the buffer distance.
buffer = geo_df.buffer(402.336)


In [36]:
# The result, buffer, is a GeoSeries where each element is a Polygon representing the area within
# a 1/4 mile of the corresponding station. You might want to add this as a column in your GeoDataFrame:

geo_df['buffer'] = buffer

In [41]:
# Optionally, convert back to latitude/longitude for compatibility with other tools
geo_df = geo_df.to_crs("EPSG:4326")


In [30]:
from google.colab import files
files.download('stations_with_buffer_bounds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
#Load your dataset
# Assumed to be in the form of a CSV with columns 'lon' and 'lat'
data = pd.read_csv('stations1.csv')

# Ensure column names are not more than 10 characters long
data.columns = [col[:10] for col in data.columns]

# Create a GeoDataFrame from your data
geometry = [Point(xy) for xy in zip(data.Lon, data.Lat)]
geo_df = gpd.GeoDataFrame(data, geometry=geometry)

#Define the CRS (coordinate reference system)
# You need to define it because operations like buffer depend on the unit of measurement,
# and lat/lon is often represented in degrees.
geo_df.crs = CRS("EPSG:4326")  # This is the CRS for latitude/longitude
geo_df = geo_df.to_crs("EPSG:3857")  # Converting to a CRS that uses meters as units (Web Mercator)

# Now, create a buffer of 1/4 mile around each point.
# Since EPSG:3857 is in meters and 1/4 mile is about 402.336 meters, we use that as the buffer distance.
buffer = geo_df.buffer(402.336)

# Replace 'geometry' with buffer
geo_df.set_geometry(buffer, inplace=True)

# Optionally, convert back to latitude/longitude for compatibility with other tools
geo_df = geo_df.to_crs("EPSG:4326")

# Save the result to a new shapefile or GeoJSON
geo_df.to_file('buffered_stations.shp')

In [46]:
from google.colab import files
files.download('buffered_stations.shp')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>